# Calibrated Explanations for Binary Classification
## Stability and Robustness

Author: Tuwe Löfström (tuwe.lofstrom@ju.se)  
Copyright 2023 Tuwe Löfström  
License: BSD 3 clause
Sources:
1. ["Calibrated Explanations: with Uncertainty Information and Counterfactuals"](https://arxiv.org/abt/2305.02305) by [Helena Löfström](https://github.com/Moffran), [Tuwe Löfström](https://github.com/tuvelofstrom), Ulf Johansson, and Cecilia Sönströd.

### 1. Import packages

In [92]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [93]:
import pickle
import numpy as np

### 2 Import results from the pickled file

In [94]:
with open('results_stab_rob.pkl', 'rb') as f:
    results = pickle.load(f)

First look at the times taken to compute the explanations.

In [95]:
s_timer = []
r_timer = []
n = results['test_size']
r = results['num_rep']

print(' & xGB & xGB & xGB & xGB & RF & RF & RF & RF \\\\')
print('Dataset & CE Stability & CCE Stability & CE Robustness & CCE Robustness  & CE Stability & CCE Stability& CE Robustness & CCE Robustness \\\\')
for d in results:
    if d in ['test_size', 'num_rep']:
        continue
    print(d, end='')
    for a in results[d]:
        print(' & ', end='')
        s_time = results[d][a]['stab_timer']
        r_time = results[d][a]['rob_timer']
        print(f'{np.mean([t/n for t in s_time["ce"]]):.2f} & {np.mean([t/n for t in s_time["cce"]]):.2f} & ',end='')
        print(f'{np.mean([t/n for t in r_time["ce"]]):.2f} & {np.mean([t/n for t in r_time["cce"]]):.2f}',end='')
    print(' \\\\')



 & xGB & xGB & xGB & xGB & RF & RF & RF & RF \\
Dataset & CE Stability & CCE Stability & CE Robustness & CCE Robustness  & CE Stability & CCE Stability& CE Robustness & CCE Robustness \\
pc1req & 0.09 & 0.09 & 0.09 & 0.09 & 0.22 & 0.22 & 0.23 & 0.23 \\


Let us fit a random forest to the proper training set. We also set a random seed to be able to rerun the notebook and get the same results.

In [98]:
from scipy import stats as st
stab_rank = {}
stab_val = {}
stab_res = {}
rob_rank = {}
rob_val = {}
rob_proba = []

n = results['test_size']
r = results['num_rep']

print(' & xGB & xGB & xGB & xGB & xGB & RF & RF & RF & RF & RF \\\\')
print('Dataset & CE Stability & CCE Stability & CE Robustness & CCE Robustness & Model Variance & CE Stability & CCE Stability & CE Robustness & CCE Robustness & Model Variance \\\\')
for d in results:
    if d in ['test_size', 'num_rep']:
        continue
    print(d, end='')
    for a in results[d]:
        print(' & ', end='')
        stability = results[d][a]['stability']
        robustness = results[d][a]['robustness']
        
        for key in ['ce', 'cce']:    
            ranks = []
            values = []
            for j in range(n):
                rank = []
                value = []
                for i in range(r):
                    rank.append(np.argsort(np.abs(stability[key][i][j]['predict']))[-1:][0])
                ranks.append(rank)
                values.append(value)
            stab_rank[key] = st.mode(ranks, axis=1)[0]
            value = []
            for j in range(n):
                value.append([np.mean([stability[key][i][j]['predict'][stab_rank[key][j][0]] for i in range(r)]), np.var([stability[key][i][j]['predict'][stab_rank[key][j][0]] for i in range(r)])])
            stab_val[key] = value 
            
            ranks = []
            values = []
            for j in range(n):
                rank = []
                value = []
                for i in range(r):
                    rank.append(np.argsort(np.abs(robustness[key][i][j]['predict']))[-1:][0])
                ranks.append(rank)
                values.append(value)
            rob_rank[key] = st.mode(ranks, axis=1)[0]
            value = []
            for j in range(n):
                value.append([np.mean([robustness[key][i][j]['predict'][rob_rank[key][j][0]] for i in range(r)]), np.var([robustness[key][i][j]['predict'][rob_rank[key][j][0]] for i in range(r)])])
            rob_val[key] = value
        
        for inst in range(n):
            rob_proba.append(np.var([robustness['proba'][j][inst] for j in range(r)]))
        
        # print(stab_rank)
        # print(stab_val)
        print(f'{np.mean([t[1] if t[1] > 1e-20 else 0 for t in stab_val["ce"]]):.1e} & {np.mean([t[1] if t[1] > 1e-20 else 0 for t in stab_val["cce"]]):.1e} & ',end='')
        print(f'{np.mean([t[1] if t[1] > 1e-20 else 0 for t in rob_val["ce"]]):.1e} & {np.mean([t[1] if t[1] > 1e-20 else 0 for t in rob_val["cce"]]):.1e} & ',end='')
        # print(f'{np.mean([t[1] for t in stab_val["ce"]]):.1e} & {np.mean([t[1] for t in stab_val["cce"]]):.1e} & ',end='')
        # print(f'{np.mean([t[1] for t in rob_val["ce"]]):.1e} & {np.mean([t[1] for t in rob_val["cce"]]):.1e} & ',end='')
        print(f'{np.mean(rob_proba):.1e}', end='')
    print(' \\\\')

 & xGB & xGB & xGB & xGB & xGB & RF & RF & RF & RF & RF \\
Dataset & CE Stability & CCE Stability & CE Robustness & CCE Robustness & Model Variance & CE Stability & CCE Stability & CE Robustness & CCE Robustness & Model Variance \\
pc1req & 7.7e-35 & 7.7e-35 & 7.7e-35 & 7.7e-35 & 7.4e-15 & 1.8e-33 & 1.8e-33 & 2.0e-03 & 2.0e-03 & 6.4e-04 \\
